# Model Iteration 1

To start looking at our dataset, we import pandas and scikit-learn to be able to properly manipulate the information. From there, we quickly use the pandas function `describe()` to show each column in the dataset.

In [56]:
import pandas

titanic = pandas.read_csv('train.csv')

print(titanic.describe())

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


Looks a bit like we already have run into a problem. All the columns consist of 891 data points, except for the age column coming in at a paltry 714 data points. To fix this, we fill in the unavailable values as the median (shown with `.fillna()` and `.median()`, so as to not skew the data or throw out otherwise perfectly usable data.

In [57]:
titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())

While we were missing some data in one of the columns before, we have also been excluding any data that was non-numerical, as it could not be easily interepreted as numbers can be. To fix this, we start by converting important data into a more readable format (in this case, binary where 1 = female and 0 = male).

In [58]:
titanic.loc[titanic["Sex"] == "male", "Sex"] = 0
titanic.loc[titanic["Sex"] == "female", "Sex"] = 1
print(titanic["Sex"])

0      0
1      1
2      1
3      1
4      0
5      0
6      0
7      0
8      1
9      1
10     1
11     1
12     0
13     0
14     1
15     1
16     0
17     0
18     1
19     1
20     0
21     0
22     1
23     0
24     1
25     1
26     0
27     0
28     1
29     0
      ..
861    0
862    1
863    1
864    0
865    1
866    1
867    0
868    0
869    0
870    0
871    1
872    0
873    0
874    1
875    1
876    0
877    0
878    0
879    1
880    1
881    0
882    1
883    0
884    0
885    1
886    0
887    1
888    1
889    0
890    0
Name: Sex, dtype: object


From the sex of the passengers, we now change the embarked column to include more information (where 0 is S, 1 is C, 2 is Q port name).

In [59]:
titanic["Embarked"] = titanic["Embarked"].fillna("S")

titanic.loc[titanic["Embarked"] == "S", "Embarked"] = 0
titanic.loc[titanic["Embarked"] == "C", "Embarked"] = 1
titanic.loc[titanic["Embarked"] == "Q", "Embarked"] = 2

Looking at all this data is quite daunting, even if it was all nicely fixed and in perfect form. However, we should also be making some guesses and links between different columns, which seems even more daunting. Well, as the saying goes, should've used machine learning! So let's do that! We'll start by creating linear regression models between three samples and predictions of our data.

In [60]:
# Sourced from dataquest tutorial

from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import KFold

predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

alg = LinearRegression()
kf = KFold(titanic.shape[0], n_folds=3, random_state=1)

predictions = []
for train, test in kf:
    train_predictors = (titanic[predictors].iloc[train,:])
    train_target = titanic["Survived"].iloc[train]
    alg.fit(train_predictors, train_target)
    test_predictions = alg.predict(titanic[predictors].iloc[test,:])
    predictions.append(test_predictions)

With possible predictions comes checks. To start, we want to check how well we did with our guesses. A really simple way of doing this is checking if values in our prediction exactly match values in the dataset.

In [61]:
import numpy as np

predictions = np.concatenate(predictions, axis=0)

predictions[predictions > .5] = 1
predictions[predictions <= .5] = 0
accuracy = sum(predictions[predictions == titanic["Survived"]]) / len(predictions)
print(accuracy)

0.783389450056


/home/philip/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index


Next step, to standardize our model a bit, is to attribute the values between 0 and 1. With this in place, we can more accurately judge predictions.

In [62]:
# Sourced from dataquest
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression

alg = LogisticRegression(random_state=1)
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic["Survived"], cv=3)
print(scores.mean())

0.787878787879


With predictions and such, we need to show off what we were able to accomplish (or what we need to work more on). To do this, we create a final train data output, and output this in a submission .csv file for processing on Kaggle.

In [63]:
titanic_test = pandas.read_csv("test.csv")
titanic_test["Age"] = titanic_test["Age"].fillna(titanic_test["Age"].median())
titanic_test.loc[titanic_test["Sex"] == "male", "Sex"] = 0 
titanic_test.loc[titanic_test["Sex"] == "female", "Sex"] = 1
titanic_test["Embarked"] = titanic_test["Embarked"].fillna("S")
titanic_test["Fare"] = titanic_test["Fare"].fillna(titanic_test["Fare"].median())

titanic_test.loc[titanic_test["Embarked"] == "S", "Embarked"] = 0
titanic_test.loc[titanic_test["Embarked"] == "C", "Embarked"] = 1
titanic_test.loc[titanic_test["Embarked"] == "Q", "Embarked"] = 2

In [64]:
# Sourced from dataquest

alg = LogisticRegression(random_state=1)

alg.fit(titanic[predictors], titanic["Survived"])

predictions = alg.predict(titanic_test[predictors])

submission = pandas.DataFrame({
        "PassengerId": titanic_test["PassengerId"],
        "Survived": predictions
    })
submission.to_csv("results.csv", index=False)

Resulting score: 	
0.75120

## Revision 1
Now to look at some more details about the data! The next logical step (considering the available columns in the data) is that of family relations, whether it be siblings, parents, or other family data. First, we start by combining the two columns (`sibsp` and `parch`) into a `family` column.

In [65]:
titanic["Family"] = titanic["SibSp"] + titanic["Parch"]
predictors = ["Pclass", "Sex", "Age", "Fare", "Embarked", "Family"]

alg = LinearRegression()
kf = KFold(titanic.shape[0], n_folds=3, random_state=1)

predictions = []
for train, test in kf:
    train_predictors = (titanic[predictors].iloc[train,:])
    train_target = titanic["Survived"].iloc[train]
    alg.fit(train_predictors, train_target)
    test_predictions = alg.predict(titanic[predictors].iloc[test,:])
    predictions.append(test_predictions)
    
alg = LogisticRegression(random_state=1)
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic["Survived"], cv=3)
print(scores.mean())

0.791245791246


Not a huge jump in our score, however that's still ~0.01 better, which is a nice plus. Let's continue with the data/exporting our values for checks.

In [66]:
titanic_test = pandas.read_csv("test.csv")
titanic_test["Age"] = titanic_test["Age"].fillna(titanic_test["Age"].median())
titanic_test.loc[titanic_test["Sex"] == "male", "Sex"] = 0 
titanic_test.loc[titanic_test["Sex"] == "female", "Sex"] = 1
titanic_test["Embarked"] = titanic_test["Embarked"].fillna("S")
titanic_test["Fare"] = titanic_test["Fare"].fillna(titanic_test["Fare"].median())

titanic_test.loc[titanic_test["Embarked"] == "S", "Embarked"] = 0
titanic_test.loc[titanic_test["Embarked"] == "C", "Embarked"] = 1
titanic_test.loc[titanic_test["Embarked"] == "Q", "Embarked"] = 2

titanic_test["Family"] = titanic_test["Parch"] + titanic_test["SibSp"]

alg = LogisticRegression(random_state=1)

alg.fit(titanic[predictors], titanic["Survived"])

predictions = alg.predict(titanic_test[predictors])

submission = pandas.DataFrame({
        "PassengerId": titanic_test["PassengerId"],
        "Survived": predictions
    })
submission.to_csv("results_family.csv", index=False)

Score! We just moved 284 positions up the leaderboard, with a new score of 0.76555!

## Revision 2
What else is interesting to look at about all this data. We have looked into embarked, family, sex, age, and fare. Considering fare, there may be less of an importance with it. If fare meant different cabins (better being near the top of the boat), there may be a big correlation, but let's see.

In [67]:
titanic["Family"] = titanic["SibSp"] + titanic["Parch"]
titanic["Fare"] = np.log(titanic["Fare"])
predictors = ["Pclass", "Sex", "Age", "Fare", "Embarked", "Family"]

alg = LinearRegression()
kf = KFold(titanic.shape[0], n_folds=3, random_state=1)

predictions = []
for train, test in kf:
    train_predictors = (titanic[predictors].iloc[train,:])
    train_target = titanic["Survived"].iloc[train]
    alg.fit(train_predictors, train_target)
    test_predictions = alg.predict(titanic[predictors].iloc[test,:])
    predictions.append(test_predictions)
    
alg = LogisticRegression(random_state=1)
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic["Survived"], cv=3)
print(scores.mean())

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

When I attempt to take the log of the fare data (to lessen it overall), the errors show that there are missing values (which there aren't) and is unable to proceed. I am unsure of why this is the case. From this point, assuming it worked, I would do the steps below to create a new Kaggle score.

In [ ]:
titanic_test = pandas.read_csv("test.csv")
titanic_test["Age"] = titanic_test["Age"].fillna(titanic_test["Age"].median())
titanic_test.loc[titanic_test["Sex"] == "male", "Sex"] = 0 
titanic_test.loc[titanic_test["Sex"] == "female", "Sex"] = 1
titanic_test["Embarked"] = titanic_test["Embarked"].fillna("S")
titanic_test["Fare"] = titanic_test["Fare"].fillna(titanic_test["Fare"].median())

titanic_test.loc[titanic_test["Embarked"] == "S", "Embarked"] = 0
titanic_test.loc[titanic_test["Embarked"] == "C", "Embarked"] = 1
titanic_test.loc[titanic_test["Embarked"] == "Q", "Embarked"] = 2

titanic_test["Family"] = titanic_test["Parch"] + titanic_test["SibSp"]

alg = LogisticRegression(random_state=1)

alg.fit(titanic[predictors], titanic["Survived"])

predictions = alg.predict(titanic_test[predictors])

submission = pandas.DataFrame({
        "PassengerId": titanic_test["PassengerId"],
        "Survived": predictions
    })
submission.to_csv("results_fare.csv", index=False)

And assuming things worked, I would be putting my updated score here.